In [ ]:
#Simple NN

In [9]:
import cv2
import numpy as np
import os

# Paths to the folders
train_folder = '/home/bhumika-avadutha/Desktop/train'
val_folder = '/home/bhumika-avadutha/Desktop/valid'
test_folder = '/home/bhumika-avadutha/Desktop/test'

# Load training data
X_train, y_train = [], []
for file in os.listdir(train_folder):
    img_path = os.path.join(train_folder, file)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_train.append(img)
        y_train.append(1)  # Assign class label 1
    else:
        print(f"Error reading file {file}")

X_train = np.array(X_train)
y_train = np.array(y_train)

# Initialize weights and bias
weights = np.random.rand(X_train.shape[1])
bias = 0

# Set learning rate and number of iterations
learning_rate = 0.01
n_iters = 1

# Train the model
for _ in range(n_iters):
    for i in range(len(X_train)):
        x_i = X_train[i]
        y_i = y_train[i]
        # Calculate the predicted output
        predicted = np.where(np.dot(x_i, weights) + bias >= 0, 1, -1)
        # Update weights and bias
        update = learning_rate * (y_i - predicted)
        weights += update * x_i
        bias += update
    
    # Calculate accuracy and loss
    predictions = np.where(np.dot(X_train, weights) + bias >= 0, 1, -1)
    accuracy = np.mean(predictions == y_train)
    loss = np.mean((predictions - y_train) ** 2)
    print(f'Iteration {_+1}, Accuracy: {accuracy:.2f}, Loss: {loss:.2f}')

# Save the trained model
np.save('weights.npy', weights)
np.save('bias.npy', bias)

# Load validation data
X_val, y_val = [], []
for file in os.listdir(val_folder):
    img_path = os.path.join(val_folder, file)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_val.append(img)
        y_val.append(1)  # Assign class label 1
    else:
        print(f"Error reading file {file}")

X_val = np.array(X_val)
y_val = np.array(y_val)

# Load the trained model
weights = np.load('weights.npy')
bias = np.load('bias.npy')

# Make predictions on validation set
predictions = np.where(np.dot(X_val, weights) + bias >= 0, 1, -1)
val_accuracy = np.mean(predictions == y_val)
print(f'Validation Accuracy: {val_accuracy:.2f}')

# Load test data
X_test, y_test = [], []
for file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, file)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (32, 32))  # Resize the image
        img = img.flatten()  # Flatten the image into a 1D array
        X_test.append(img)
        y_test.append(1)  # Assign class label 1
    else:
        print(f"Error reading file {file}")

X_test = np.array(X_test)
y_test = np.array(y_test)

# Make predictions on testing set
predictions = np.where(np.dot(X_test, weights) + bias >= 0, 1, -1)
test_accuracy = np.mean(predictions == y_test)
print(f'Testing Accuracy: {test_accuracy:.2f}')

Iteration 1, Accuracy: 1.00, Loss: 0.00
Validation Accuracy: 1.00
Testing Accuracy: 1.00


In [ ]:
#Regularization

In [10]:
import numpy as np
import os 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


train_directory = "/home/bhumika-avadutha/Desktop/train"
valid_directory = "/home/bhumika-avadutha/Desktop/valid"
test_directory = "/home/bhumika-avadutha/Desktop/test"
image_width, image_height = 28, 28  # Image dimensions
batch_size = 32
epochs = 10

# Initialize lists to hold data and labels
train_data, train_labels = [], []
valid_data, valid_labels = [], []
test_data, test_labels = [], []

# Load and preprocess images for training
directory = train_directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(image_width, image_height))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        train_data.append(img_array)
        train_labels.append(0)  # Assuming all images are of the same class (e.g., class 0)

# Load and preprocess images for validation
directory = valid_directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(image_width, image_height))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        valid_data.append(img_array)
        valid_labels.append(0)  # Assuming all images are of the same class (e.g., class 0)

# Load and preprocess images for testing
directory = test_directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(image_width, image_height))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        test_data.append(img_array)
        test_labels.append(0)  # Assuming all images are of the same class (e.g., class 0)

# Convert lists to numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Print number of images found
print(f"Found {len(train_data)} images for training.")
print(f"Found {len(valid_data)} images for validation.")
print(f"Found {len(test_data)} images for testing.")

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Set up data generator for training
train_generator = train_datagen.flow(
    x=train_data,
    y=train_labels,
    batch_size=batch_size,
    shuffle=True
)

valid_generator = valid_datagen.flow(
    x=valid_data,
    y=valid_labels,
    batch_size=batch_size,
    shuffle=False  # No need to shuffle validation data
)

train_accuracies = []
valid_accuracies = []
test_accuracies = []

# Model building and training with L2 regularization
print("Model with L2 regularization:")
model_l2 = Sequential([
    Flatten(input_shape=(image_width, image_height, 3)),  # input layer
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # hidden layer with L2 regularization
    BatchNormalization(),  # batch normalization layer
    Dropout(0.5),  # dropout layer for regularization
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # additional hidden layer with L2 regularization
    BatchNormalization(),  # batch normalization layer
    Dropout(0.3),  # dropout layer for regularization
    Dense(1, activation='sigmoid')  # output layer, sigmoid for binary classification
])

model_l2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model with L2 regularization
history_l2 = model_l2.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[early_stopping]
)

# Evaluate the model with L2 regularization on training data
train_loss_l2, train_accuracy_l2 = model_l2.evaluate(train_generator, steps=len(train_generator))

# Evaluate the model with L2 regularization on validation data
valid_loss_l2, valid_accuracy_l2 = model_l2.evaluate(valid_generator, steps=len(valid_generator))

# Evaluate the model with L2 regularization on testing data
test_loss_l2, test_accuracy_l2 = model_l2.evaluate(test_data, test_labels)

print(f'Training accuracy: {train_accuracy_l2}')
print(f'Validation accuracy: {valid_accuracy_l2}')
print(f'Testing accuracy with L2 regularization: {test_accuracy_l2}')

# Store accuracies
train_accuracies.append(train_accuracy_l2)
valid_accuracies.append(valid_accuracy_l2)
test_accuracies.append(test_accuracy_l2)

# Model building and training with Dropout
print("\nModel with Dropout:")
model_dropout = Sequential([
    Flatten(input_shape=(image_width, image_height, 3)),  # input layer
    Dense(128, activation='relu'),  # hidden layer
    BatchNormalization(),  # batch normalization layer
    Dropout(0.5),  # dropout layer
    Dense(64, activation='relu'),  # additional hidden layer
    BatchNormalization(),  # batch normalization layer
    Dropout(0.3),  
    Dense(1, activation='sigmoid')  # output layer, sigmoid for binary classification
])

model_dropout.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with Dropout
history_dropout = model_dropout.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[early_stopping]
)

# Evaluate the model with Dropout on training data
train_loss_dropout, train_accuracy_dropout = model_dropout.evaluate(train_generator, steps=len(train_generator))

# Evaluate the model with Dropout on validation data
valid_loss_dropout, valid_accuracy_dropout = model_dropout.evaluate(valid_generator, steps=len(valid_generator))

# Evaluate the model with Dropout on testing data
test_loss_dropout, test_accuracy_dropout = model_dropout.evaluate(test_data, test_labels)


print(f'Training accuracy: {train_accuracy_dropout}')
print(f'Validation accuracy: {valid_accuracy_dropout}')
print(f'Testing accuracy with Dropout: {test_accuracy_dropout}')


# Store accuracies
train_accuracies.append(train_accuracy_dropout)
valid_accuracies.append(valid_accuracy_dropout)
test_accuracies.append(test_accuracy_dropout)

# Model building and training with Batch Normalization
print("\nModel with Batch Normalization:")
model_batchnorm = Sequential([
    Flatten(input_shape=(image_width, image_height, 3)),  
    Dense(128, activation='relu'),  # hidden layer
    BatchNormalization(),  # batch normalization layer
    Dropout(0.5),  # dropout layer
    Dense(64, activation='relu'), 
    BatchNormalization(), 
    Dropout(0.3),  
    Dense(1, activation='sigmoid')  # output layer, sigmoid for binary classification
])

model_batchnorm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with Batch Normalization
history_batchnorm = model_batchnorm.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[early_stopping]
)

# Evaluate the model with Batch Normalization on training data
train_loss_batchnorm, train_accuracy_batchnorm = model_batchnorm.evaluate(train_generator, steps=len(train_generator))

# Evaluate the model with Batch Normalization on validation data
valid_loss_batchnorm, valid_accuracy_batchnorm = model_batchnorm.evaluate(valid_generator, steps=len(valid_generator))

# Evaluate the model with Batch Normalization on testing data
test_loss_batchnorm, test_accuracy_batchnorm = model_batchnorm.evaluate(test_data, test_labels)
print(f'Training accuracy: {train_accuracy_batchnorm}')
print(f'Validation accuracy: {valid_accuracy_batchnorm}')
print(f'Testing accuracy with Batch Normalization: {test_accuracy_batchnorm}')

# Store accuracies
train_accuracies.append(train_accuracy_batchnorm)
valid_accuracies.append(valid_accuracy_batchnorm)
test_accuracies.append(test_accuracy_batchnorm)

# Model building and training with Early Stopping

print("\nModel with Early Stopping:")
model_earlystop = Sequential([
    Flatten(input_shape=(image_width, image_height, 3)),  # input layer
    Dense(128, activation='relu'),  # hidden layer
    BatchNormalization(),  # batch normalization layer
    Dropout(0.5),  # dropout layer
    Dense(64, activation='relu'),  # additional hidden layer
    BatchNormalization(),  # batch normalization layer
    Dropout(0.3),  # dropout layer
    Dense(1, activation='sigmoid')  # output layer, sigmoid for binary classification
])

model_earlystop.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with Early Stopping
history_earlystop = model_earlystop.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[early_stopping]
)

# Evaluate the model with Early Stopping on training data
train_loss_earlystop, train_accuracy_earlystop = model_earlystop.evaluate(train_generator, steps=len(train_generator))

# Evaluate the model with Early Stopping on validation data
valid_loss_earlystop, valid_accuracy_earlystop = model_earlystop.evaluate(valid_generator, steps=len(valid_generator))

# Evaluate the model with Early Stopping on testing data
test_loss_earlystop, test_accuracy_earlystop = model_earlystop.evaluate(test_data, test_labels)

print(f'Training accuracy: {train_accuracy_earlystop}')
print(f'Validation accuracy: {valid_accuracy_earlystop}')
print(f'Testing accuracy with Early Stopping: {test_accuracy_earlystop}')

# Store accuracies
train_accuracies.append(train_accuracy_batchnorm)
valid_accuracies.append(valid_accuracy_batchnorm)
test_accuracies.append(test_accuracy_batchnorm)

# Print final accuracies
print("\nFinal accuracies:")
print(f'L2 Regularization - Training: {train_accuracies[0]}, Validation: {valid_accuracies[0]}, Testing: {test_accuracies[0]}')
print(f'Dropout - Training: {train_accuracies[1]}, Validation: {valid_accuracies[1]}, Testing: {test_accuracies[1]}')
print(f'Batch Normalization - Training: {train_accuracies[2]}, Validation: {valid_accuracies[2]}, Testing: {test_accuracies[2]}')
print(f'Early Stopping - Training: {train_accuracies[3]}, Validation: {valid_accuracies[3]}, Testing: {test_accuracies[3]}')


Found 320 images for training.
Found 40 images for validation.
Found 40 images for testing.
Model with L2 regularization:
Epoch 1/10


/home/bhumika-avadutha/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.4824 - loss: 1.3354 - val_accuracy: 0.3750 - val_loss: 1.0442
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


2024-08-20 11:38:51.946934: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-08-20 11:38:52.010362: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/bhumika-avadutha/.local/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5281 - loss: 1.1493 - val_accuracy: 0.5500 - val_loss: 1.0005
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5887 - loss: 1.0681  

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6025 - loss: 0.9854 - val_accuracy: 1.0000 - val_loss: 0.7095
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.6690
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.7082 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6583 - loss: 11.6514 
Training accuracy: 1.0
Validation accuracy: 1.0
Testing accuracy with L2 regularization: 0.675000011920929

Model with Dropout:
Epoch 1/10


/home/bhumika-avadutha/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.5226 - loss: 0.9402 - val_accuracy: 0.2250 - val_loss: 0.7899
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5903 - loss: 0.7497 - val_accuracy: 0.5000 - val_loss: 0.6893
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6065 - loss: 0.7958 - val_accuracy: 0.9250 - val_loss: 0.4963
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6730 - loss: 0.6574 - val_accuracy: 0.7750 - val_loss: 0.5795
Epoch 8/10


2024-08-20 11:38:55.435952: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6318 - loss: 0.6702 - val_accuracy: 0.5000 - val_loss: 0.7292
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9646 - loss: 0.4463
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9187 - loss: 0.5024 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7604 - loss: 11.3398 
Training accuracy: 0.9750000238418579
Validation accuracy: 0.925000011920929
Testing accuracy with Dropout: 0.75

Model with Batch Normalization:
Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5423 - loss: 0.8953 - val_accuracy: 0.9750 - val_loss: 0.4930
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5351 - loss: 0.8582 - val_accuracy: 0.9750 - val_l

In [ ]:
#Perceptron

In [11]:
import os
import numpy as np
from PIL import Image

# Directory paths
train_dir = '/home/bhumika-avadutha/Desktop/train/'
valid_dir = '/home/bhumika-avadutha/Desktop/valid/'
test_dir = '/home/bhumika-avadutha/Desktop/test/'

epochs = 1

num_features = None
weights = None
bias = None
learning_rate = 0.1

# Training and evaluation in a single loop
for epoch in range(epochs):
    # Read training data
    X_train = []
    y_train = []
    for filename in os.listdir(train_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as per your images
            file_path = os.path.join(train_dir, filename)
            img = Image.open(file_path)
            img = img.resize((100, 100)).convert('L')  # Resize and convert to grayscale ('L' mode)
            img_array = np.array(img)
            X_train.append(img_array.flatten())  # Flatten image into 1D array
            # Assuming the label is encoded in the filename or directory structure
            label = 1 if 'class1' in filename else 0  # Adjust based on your data
            y_train.append(label)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Check if X_train is empty or has unexpected structure
    if X_train.shape[0] == 0 or X_train.ndim != 2 or X_train.shape[1] < 1:
        print(f"Warning: X_train is empty or has unexpected structure in epoch {epoch + 1}. Skipping epoch.")
        continue
    
    # Initialize weights and bias on first epoch
    if epoch == 0:
        num_features = X_train.shape[1]
        weights = np.zeros(num_features)
        bias = 0.0
    
    # Read validation data
    X_valid = []
    y_valid = []
    for filename in os.listdir(valid_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as per your images
            file_path = os.path.join(valid_dir, filename)
            img = Image.open(file_path)
            img = img.resize((100, 100)).convert('L')  # Resize and convert to grayscale ('L' mode)
            img_array = np.array(img)
            X_valid.append(img_array.flatten())  # Flatten image into 1D array
            # Assuming the label is encoded in the filename or directory structure
            label = 1 if 'class1' in filename else 0  # Adjust based on your data
            y_valid.append(label)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)
    
    # Read test data
    X_test = []
    y_test = []
    for filename in os.listdir(test_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as per your images
            file_path = os.path.join(test_dir, filename)
            img = Image.open(file_path)
            img = img.resize((100, 100)).convert('L')  # Resize and convert to grayscale ('L' mode)
            img_array = np.array(img)
            X_test.append(img_array.flatten())  # Flatten image into 1D array
            # Assuming the label is encoded in the filename or directory structure
            label = 1 if 'class1' in filename else 0  # Adjust based on your data
            y_test.append(label)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    # Training loop
    correct_train = 0
    for i in range(len(X_train)):
        x = X_train[i]
        y_true = y_train[i]
        
        # Predict
        linear_output = np.dot(weights, x) + bias
        y_pred = 1 if linear_output >= 0 else 0
        
        # Update weights and bias if prediction is incorrect
        if y_true != y_pred:
            if y_true == 1:
                weights += learning_rate * x
                bias += learning_rate
            else:
                weights -= learning_rate * x
                bias -= learning_rate
        
        # Calculate training accuracy
        if y_true == y_pred:
            correct_train += 1
        train_accuracy = correct_train / len(X_train)
    print(f"train accuracy ={train_accuracy:.2%}")   
    # Validation accuracy
    correct_valid = 0
    for i in range(len(X_valid)):
        x = X_valid[i]
        y_true = y_valid[i]
        linear_output = np.dot(weights, x) + bias
        y_pred = 1 if linear_output >= 0 else 0
        if y_true == y_pred:
            correct_valid += 1
    validation_accuracy = correct_valid / len(X_valid)
    
# print(f"Epoch {epoch + 1}: Train Accuracy = {train_accuracy:.2%}, Validation Accuracy = {validation_accuracy:.2%}")

    print(f"validate accuracy = {validation_accuracy:.2%}")


# Test accuracy
    correct_test = 0
    for i in range(len(X_test)):
       x = X_test[i]
       y_true = y_test[i]
       linear_output = np.dot(weights, x) + bias
       y_pred = 1 if linear_output >= 0 else 0
       if y_true == y_pred:
        correct_test += 1
    test_accuracy = correct_test / len(X_test)

    print(f"Test Accuracy = {test_accuracy:.2%}")


train accuracy =99.69%
validate accuracy = 100.00%
Test Accuracy = 100.00%


In [ ]:
#Optimizers

In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


image_width = 150
image_height = 150
epochs = 10 
# Define optimizers to compare
optimizers = {
    'adam': 'adam',
    'sgd': SGD(),
    'sgd_momentum': SGD(momentum=0.9),
    'rmsprop': RMSprop(learning_rate=0.001)  
}

# Initialize lists to hold accuracies for each optimizer
train_accuracies_opt = {opt_name: [] for opt_name in optimizers}
valid_accuracies_opt = {opt_name: [] for opt_name in optimizers}
test_accuracies_opt = {opt_name: [] for opt_name in optimizers}


train_dir = '/home/bhumika-avadutha/Desktop/train'
valid_dir = '/home/bhumika-avadutha/Desktop/valid'
test_dir = '/home/bhumika-avadutha/Desktop/test'

# Load and preprocess data for each dataset
for opt_name, optimizer in optimizers.items():
    print(f"\nTraining model with optimizer: {opt_name}")
    
    # Load training data
    train_images = []
    train_labels = []
    train_files = [os.path.join(train_dir, file) for file in os.listdir(train_dir)]
    for file in train_files:
        img = load_img(file, target_size=(image_width, image_height))
        img_array = img_to_array(img) / 255.0
        train_images.append(img_array)
        train_labels.append(1 if 'positive' in file else 0)  # Adjust label based on naming convention
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)

    # Load validation data
    valid_images = []
    valid_labels = []
    valid_files = [os.path.join(valid_dir, file) for file in os.listdir(valid_dir)]
    for file in valid_files:
        img = load_img(file, target_size=(image_width, image_height))
        img_array = img_to_array(img) / 255.0
        valid_images.append(img_array)
        valid_labels.append(1 if 'positive' in file else 0)  # Adjust label based on naming convention
    valid_images = np.array(valid_images)
    valid_labels = np.array(valid_labels)

    # Load testing data
    test_images = []
    test_labels = []
    test_files = [os.path.join(test_dir, file) for file in os.listdir(test_dir)]
    for file in test_files:
        img = load_img(file, target_size=(image_width, image_height))
        img_array = img_to_array(img) / 255.0
        test_images.append(img_array)
        test_labels.append(1 if 'positive' in file else 0)  # Adjust label based on naming convention
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    # Model definition
    model = Sequential([
        Flatten(input_shape=(image_width, image_height, 3)),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_images, train_labels,
        epochs=epochs,
        batch_size=20,  # Mini-batch size
        validation_data=(valid_images, valid_labels),
        callbacks=[early_stopping]
    )

    # Evaluate the model on training data
    train_loss, train_accuracy = model.evaluate(train_images, train_labels)
    train_accuracies_opt[opt_name].append(train_accuracy)

    # Evaluate the model on validation data
    valid_loss, valid_accuracy = model.evaluate(valid_images, valid_labels)
    valid_accuracies_opt[opt_name].append(valid_accuracy)

    # Evaluate the model on testing data
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    test_accuracies_opt[opt_name].append(test_accuracy)

    # Print accuracies
    print(f'Training accuracy: {train_accuracy}')
    print(f'Validation accuracy: {valid_accuracy}')
    print(f'Testing accuracy: {test_accuracy}')

# Print final results
for opt_name in optimizers:
    print(f"\nOptimizer: {opt_name}")
    print(f"Training accuracy: {train_accuracies_opt[opt_name]}")
    print(f"Validation accuracy: {valid_accuracies_opt[opt_name]}")
    print(f"Testing accuracy: {test_accuracies_opt[opt_name]}")



Training model with optimizer: adam
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.5386 - loss: 0.8402 - val_accuracy: 0.7250 - val_loss: 0.7087
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.5418 - loss: 0.7398 - val_accuracy: 0.6000 - val_loss: 1.0652
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.5521 - loss: 0.6863 - val_accuracy: 0.5250 - val_loss: 0.8852
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 138ms/step - accuracy: 0.7161 - loss: 0.5720 - val_accuracy: 0.7250 - val_loss: 0.5924
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step - accuracy: 0.6752 - loss: 0.5943 - val_accuracy: 0.7000 - val_loss: 0.5770
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - accuracy: 0.7492 - loss: 0.5216 - val_accuracy: 0.9000 - val_loss: 0.4420
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.8182 - loss: 0.4230 - val_accuracy: 0.9500 - val_loss: 0.3609
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.8

In [ ]:
#Multilayer perceptron

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np

# Load MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define input shape and number of classes
input_shape = train_images[0].shape
num_classes = 10  # Since MNIST has 10 classes (digits 0-9)

# Build the model without dropout
model = Sequential([
    Flatten(input_shape=input_shape),           # Flatten input to 1D
    Dense(512, activation='relu'),              # Hidden layer with 512 neurons, ReLU activation
    Dense(256, activation='relu'),              
    Dense(128, activation='relu'),              
    Dense(num_classes, activation='softmax')    # Output layer with softmax activation for classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),   # Optimizer with lower learning rate
              loss=SparseCategoricalCrossentropy(),  # Loss function for multi-class classification
              metrics=['accuracy'])                 # Metrics to monitor during training

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Training
history = model.fit(train_images, train_labels,
                    epochs=50,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f'Test accuracy: {test_acc:.4f}')

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6978 - loss: 1.1331 - val_accuracy: 0.9296 - val_loss: 0.2554
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9344 - loss: 0.2366 - val_accuracy: 0.9502 - val_loss: 0.1817
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9509 - loss: 0.1694 - val_accuracy: 0.9554 - val_loss: 0.1556
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9622 - loss: 0.1335 - val_accuracy: 0.9550 - val_loss: 0.1491
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9674 - loss: 0.1136 - val_accuracy: 0.9637 - val_loss: 0.1221
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9729 - loss: 0.0947 - val_accuracy: 0.9647 - val_loss: 0.1155
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9785 - loss: 0.0782 - val_accuracy: 0.9682 - val_loss: 0.1080
Epoch 8/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9817 - loss: 0.0647 - val_accuracy: 0